In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import firedrake
import icepack, icepack.models, icepack.plot
from icepack.constants import ice_density as ρ_I, water_density as ρ_W, gravity as g
import numpy as np

In [ ]:
Lx, Ly = 1e3, 1e3
nx, ny = 20, 20
mesh2d = firedrake.RectangleMesh(nx, ny, Lx, Ly)
mesh = firedrake.ExtrudedMesh(mesh2d, layers=1)

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2, vfamily='DG', vdegree=0)
V = firedrake.VectorFunctionSpace(mesh, dim=2, family='CG', degree=2, vfamily='GL', vdegree=6)
Q3D = firedrake.FunctionSpace(mesh, family='DG', degree=2, vfamily='GL', vdegree=1+V.ufl_element().degree()[1])

In [ ]:
x, y, ζ = firedrake.SpatialCoordinate(mesh)

In [ ]:
b = firedrake.interpolate(firedrake.Constant(0.0), Q)
s = firedrake.interpolate(firedrake.Constant(1000), Q)
h = firedrake.interpolate(s - b, Q)

ux = firedrake.Constant(40)
uy = firedrake.Constant(0)
uz = firedrake.Constant(-40)
u = firedrake.interpolate(firedrake.as_vector((ux, uy)), V)
w = firedrake.interpolate(uz/h,Q3D)

num_years = 10.0

In [ ]:
# create a mask assuming age propogation from upper left to lower right
x_eval_min = 0 + ux.dat.data[0] * num_years
#x_eval_max = Lx
y_eval_min = 0 + uy.dat.data[0] * num_years
#y_eval_max = Ly
#z_eval_min = 0
z_eval_max = (h.dat.data[0] + uz.dat.data[0] * num_years) / h.dat.data[0]

#if (x_eval_max <= x_eval_min) or (y_eval_max <= y_eval_min) or (z_eval_max <= z_eval_min):
#    raise ValueError('No points to evaluate.')
    
mask_x = firedrake.conditional(x<x_eval_min,0,1)
mask_y = firedrake.conditional(y<y_eval_min,0,1)
mask_z = firedrake.conditional(ζ>z_eval_max,0,1)
mask = mask_x * mask_y * mask_z

fig, axes = icepack.plot.subplots()
colors = icepack.plot.tripcolor(firedrake.interpolate(mask, Q3D), axes=axes)
fig.colorbar(colors, label='mask')

In [ ]:
def velocity_3D(**kwargs):
    u = kwargs['velocity']
    h = kwargs['thickness']
    uz = kwargs['uz']
    
    Q = h.function_space()
    V = u.function_space()
    mesh = Q.mesh()
    xdegree_u, zdegree_u = u.ufl_element().degree()
    W = firedrake.FunctionSpace(mesh, family='CG', degree=2, vfamily='GL', vdegree=1+V.ufl_element().degree()[1])
    w = firedrake.interpolate(uz/h,W)
    V3D = firedrake.VectorFunctionSpace(mesh, dim=3, family='CG', degree=xdegree_u, vfamily='GL',vdegree=zdegree_u+1)
    u3D = firedrake.Function(V3D).interpolate(firedrake.as_vector((u[0],u[1],w)))
    
    return u3D

In [ ]:
u3D = velocity_3D(velocity=u, thickness=h, uz=uz)

In [ ]:
age_model = icepack.models.AgeTransport(velocity_3D=velocity_3D)
age_solver = icepack.solvers.AgeSolver(age_model)

In [ ]:
import tqdm
import numpy as np

x0=0
y0=0
z0=900
age0=0

timesteps_per_year = [5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

L1s = []

for tpy in timesteps_per_year:
    
    age = firedrake.interpolate(firedrake.Constant(0.0),Q3D)

    δt = 1.0 / tpy
    num_timesteps = int(num_years * tpy)
    
    num_age = np.zeros((num_timesteps,1))*np.nan
    ana_age = np.zeros((num_timesteps,1))*np.nan
    ts = np.zeros((num_timesteps,1))*np.nan

    for step in tqdm.trange(num_timesteps):
        age = age_solver.solve(δt,velocity=u,thickness=h,age=age,uz=uz)
        t = step*δt
        ts[step] = t
        num_age[step] = age.at(x0+ux*t,y0+uy*t,(z0+uz*t)/1000,tolerance=1e-10)
        ana_age[step] = age0+t
    
    num_sol = firedrake.interpolate(age*mask, Q3D)
    ana_sol = firedrake.interpolate((age0+t)*mask, Q3D)

    J = (num_sol-ana_sol) * firedrake.dx
    L1 = firedrake.assemble(J)
    L1s += [L1]
    print(tpy, L1)
    
    #plt.figure()
    #plt.plot(ts,num_age,label="numerical")
    #plt.plot(ts,ana_age,label="analytical")
    #plt.legend()

    plt.figure()
    plt.plot(ts,num_age-ana_age,label="numerical-analytical")
    plt.legend()
    plt.title(str(tpy))

In [ ]:
plt.plot(1/np.array(timesteps_per_year),L1s,'ko')

In [ ]:
print(num_sol.dat.data.max())
print(ana_sol.dat.data.max())

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

nz=20

xs = np.array([(Lx * k / nx, 0, .5) for k in range(nx + 1)])
ss = np.array(s.at(xs, tolerance=1e-10))
hs = np.array(h.at(xs, tolerance=1e-10))
bs = np.array(b.at(xs, tolerance=1e-10))

xz = np.array([(Lx * k / nx, 0, i)  for k in range(nx + 1) for i in np.arange(0,1.0,1.0/nz)])
ageM= np.array(age.at(xz, tolerance=1e-10))
ageM=np.reshape(ageM,(21,nz));
xz=np.reshape(xz,(21,nz,3));
hz=np.reshape(np.repeat(hs,nz),(21,nz));
hz=np.multiply(xz[:,:,2],hz);

In [ ]:
fig, axs = plt.subplots()
for i in range(0,nz):
    points = np.array([np.squeeze(xz[:,i,0]),np.squeeze(ss-hz[:,i])]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    norm = plt.Normalize(0,20)
    lc = LineCollection(segments, cmap='RdBu', norm=norm)
    # Set the values used for colormapping
    lc.set_array(np.squeeze(ageM[:,nz-1-i]))
    lc.set_linewidth(2)
    line = axs.add_collection(lc)
axs.set_xlim(xz.min(), xz.max())
axs.set_ylim(bs.min(), ss.max())
axs.plot(xs[:, 0], bs, color='black')
fig.colorbar(line, ax=axs,label=r'age $(years)$')
axs.set_ylabel('elevation (m)')
axs.set_xlabel('distance (m)')
plt.show()

In [ ]:
fig, axes = icepack.plot.subplots()
colors = icepack.plot.tripcolor(firedrake.interpolate(num_sol-ana_sol, Q3D), axes=axes)
fig.colorbar(colors, label='masked age')